## Prueba de Desempeño en Algoritmo de multiplicación de matrices cuadradas
Versión secuencial VS Versión Concurrente

### Selección de Método para medir el tiempo de CPU de Usuario

Primero, queremos evaluar algunas funciones para medir el tiempo y saber cuál es más precisa para determinar el tiempo de *CPU de usuario*

In [16]:
from sys import argv
import random, time
from resource import getrusage as resource_usage, RUSAGE_SELF
# from time import time as timestamp

#n = int(argv[1])
n= 100

def reset_to_0(the_array):
    for i, e in enumerate(the_array):
        if isinstance(e, list):
            reset_to_0(e)
        else:
            the_array[i] = 0

#Create matrix
a =[]
b =[]
r= []
matrix1 =[]
matrix2 =[]
result = []

for i in range(n):
    for i in range(n):
        number = random.randrange(10)
        number2 = random.randrange(10)
        a.append(number)
        b.append(number2)
        r.append(0)
    matrix1.append(a)
    matrix2.append(b)
    result.append(r)
    a = []
    b = []
    r = []

#Prueba de verificacion. No se recomienda su ejecucion para n -> oo
# print('Matrix 1')
# for m in matrix1:
#     print(m)

# print('Matrix 2')
# for m in matrix2:
#     print(m)

#### Prueba 1 algoritmo secuencial usando time.perf_counter()


In [14]:
#Multiply Matrix
tic = time.perf_counter()
#iterate through rows of matrix1
for i in range(n):
   # iterate through columns of matrix2
   for j in range(n):
       # iterate through rows of matrix2
       for k in range(n):
           result[i][j] += matrix1[i][k] * matrix2[k][j]

toc = time.perf_counter()
print(f"Matrix multiplication in {toc - tic:0.12f} seconds")


print('Result:')
# for r in result:
#    print(r)

Matrix multiplication in 0.284225500000 seconds
Result:
[1487, 1587, 1566, 1665, 1746, 1509, 1651, 1985, 1653, 1640, 1504, 1444, 1782, 1415, 1655, 1633, 1639, 1775, 1596, 1729, 1597, 1787, 1711, 1694, 1760, 1414, 1767, 1467, 1893, 1607, 1714, 1508, 1428, 1830, 1804, 1519, 1634, 1712, 1843, 1556, 1797, 1878, 1818, 1749, 1409, 1430, 1795, 1627, 1799, 1445, 1794, 1595, 1444, 1426, 1578, 1801, 1617, 1702, 1840, 1655, 1700, 1576, 1749, 1975, 1827, 1412, 1587, 1782, 1592, 1655, 1621, 1738, 1708, 1728, 1758, 1510, 1850, 1614, 1705, 1528, 1581, 1806, 1707, 1782, 1795, 1666, 1626, 1758, 1678, 1804, 1664, 1517, 1618, 1779, 1587, 1676, 1395, 1704, 1743, 1506]
[2055, 1954, 2092, 2012, 2096, 1577, 2100, 2398, 2081, 2032, 2060, 1990, 2256, 1883, 1979, 2107, 2055, 2043, 1983, 2147, 2191, 2083, 2144, 1993, 2153, 2088, 2055, 2184, 2272, 1859, 2213, 2212, 2055, 2215, 2152, 1963, 2094, 2159, 2070, 1868, 2005, 1926, 2358, 2078, 1946, 1903, 1960, 2133, 2129, 2107, 2303, 1915, 1838, 2125, 1814, 2271, 1926, 

#### Prueba 2 algoritmo secuencial usando resource_usage
 'real': end_time - start_time, \
 'sys': end_resources.ru_stime - start_resources.ru_stime, 
 ##### 'user': end_resources.ru_utime - start_resources.ru_utime

In [15]:
reset_to_0(result)
#Multiply Matrix
start_resources = resource_usage(RUSAGE_SELF)
#iterate through rows of matrix1
for i in range(n):
   # iterate through columns of matrix2
   for j in range(n):
       # iterate through rows of matrix2
       for k in range(n):
           result[i][j] += matrix1[i][k] * matrix2[k][j]
end_resources = resource_usage(RUSAGE_SELF)

print(f"Matrix multiplication in {end_resources.ru_utime - start_resources.ru_utime:0.32f} seconds")

print('Result:')
for r in result:
   print(r)

Matrix multiplication in 0.28125000000000000000000000000000 seconds
Result:
[1487, 1587, 1566, 1665, 1746, 1509, 1651, 1985, 1653, 1640, 1504, 1444, 1782, 1415, 1655, 1633, 1639, 1775, 1596, 1729, 1597, 1787, 1711, 1694, 1760, 1414, 1767, 1467, 1893, 1607, 1714, 1508, 1428, 1830, 1804, 1519, 1634, 1712, 1843, 1556, 1797, 1878, 1818, 1749, 1409, 1430, 1795, 1627, 1799, 1445, 1794, 1595, 1444, 1426, 1578, 1801, 1617, 1702, 1840, 1655, 1700, 1576, 1749, 1975, 1827, 1412, 1587, 1782, 1592, 1655, 1621, 1738, 1708, 1728, 1758, 1510, 1850, 1614, 1705, 1528, 1581, 1806, 1707, 1782, 1795, 1666, 1626, 1758, 1678, 1804, 1664, 1517, 1618, 1779, 1587, 1676, 1395, 1704, 1743, 1506]
[2055, 1954, 2092, 2012, 2096, 1577, 2100, 2398, 2081, 2032, 2060, 1990, 2256, 1883, 1979, 2107, 2055, 2043, 1983, 2147, 2191, 2083, 2144, 1993, 2153, 2088, 2055, 2184, 2272, 1859, 2213, 2212, 2055, 2215, 2152, 1963, 2094, 2159, 2070, 1868, 2005, 1926, 2358, 2078, 1946, 1903, 1960, 2133, 2129, 2107, 2303, 1915, 1838, 2125